# Fonctions pour le calcul avec du Bayésien naïf pour les variables catégorielles

## Lecture et nettoyage des données (code fourni)

In [5]:
using CSV, DataFrames, Gadfly, Distributions, StatsBase

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly C:\Users\Antoine Lamontagne\.julia\packages\Gadfly\09PWZ\src\mapping.jl:228


In [22]:
train = CSV.read("ODtrain.csv")

dropmissing!(train, disallowmissing=true)

n = size(train,1)
p = size(train,2)

str_mode = ["Voiture", "Transport Collectif", "Hybride", "Actif", "Autre"]

d = Array{String}(undef,n)

for i=1:length(str_mode)
   
    ind = train[:D_Mode] .== i
    d[ind] .= str_mode[i] 
    
end

train[:D_Mode_str] = d

categorical!(train, :D_Mode_str);
first(train,50)

,IPERE,Menage,M_AUTO,M_PERS,M_DOMSM,M_revenu,M_dist_train,M_dist_metro,M_dist_centre,M_emploiTC,P_RANG,P_SEXE,P_AGE,P_GRAGE,P_STATUT,P_PERMIS,D_HREDE,D_MOTIF,D_Mode,D_Mode_str
,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Categorical…
1,9,1000013,2,3,110,6,7.71979,1.81961,9.44328,556670,2,1,47,10,1,1,730,1,1,Voiture
2,17,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,1,1,46,10,3,1,630,4,2,Transport Collectif
3,21,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,3,1,47,10,1,2,500,1,2,Transport Collectif
4,31,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,7,1,21,5,3,2,700,4,2,Transport Collectif
5,29,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,6,2,54,11,1,1,630,1,1,Voiture
6,25,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,5,1,56,12,1,1,600,1,1,Voiture
7,23,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,4,1,63,13,1,2,700,1,4,Actif
8,19,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,2,2,15,4,3,5,750,4,4,Actif
9,39,1000016,0,1,406,2,3.12452,0.689126,12.9415,658600,1,1,32,7,1,1,830,1,2,Transport Collectif


## Code provenant du fichier Maximum de la vraisemblance
À mettre dans une catégorie à part dans le rapport plz

In [7]:
## COPIED FROM MAX VRAISEMBLANCE
k=5
struct TestSet
    train::DataFrame
    test::DataFrame
end

function getTestSet(initialSet::DataFrame)
    n = length(initialSet[:D_Mode])
    nTest = convert(Int64, round(0.20 * n))
    index = rand(0:n-nTest+1)
    return TestSet(vcat(initialSet[1:index-1, :], initialSet[index+nTest:n, :]), initialSet[index:index+nTest-1, :])
    
    a = [1,2,3]
    b= view(a, 1:2)
    c= vcat(view(a, 1:1), view(a, 3:3))
end
function validate(estimates::Array{Int64}, realValue::Array{Int64})
    n = length(realValue)
    nEach = [0, 0, 0, 0, 0]
    right = [0, 0, 0, 0, 0]
    for i=1:n
        nEach[realValue[i]] += 1
        if estimates[i] == realValue[i]
            right[realValue[i]] += 1
        end
    end
    
    out = ""
    for i=1:k
        #print(str_mode[i], " : ", right[i]/nEach[i]*100, "%\n" ) 
        out *= str_mode[i] * " : " * string(right[i]/nEach[i]*100) * "%\n"
    end
    #print("Total : ", sum(right)/n*100, "%\n" )
    tot = sum(right)/n*100
    out *= "Total : " * string(tot) * "%\n"
    return (total=tot, output=out)
end

validate (generic function with 1 method)

## Fonctions et structures pour le calcul

### Structure qui contient les nombres d'occurences des données nécessaires

##### Occurences:
Cette structure contient les occurences des différentes variables.<br>
- mode[i] contient ni, soit le nombre de déplacements qui ont le mode de transport i
- variables[i, j][l] contient le nombre de déplacements qui ont le mode de transport i et qui ont la catégorie l pour la variable j. Ex: Si la permière variable est P_PERMIS, variables[4, 1][2] est le nombre de déplacement où la personne ne possède pas un permis de conduire et où le mode de transport est actif
- columns[j] contient les symboles associés aux colonnes qui contiennent les variables j
- nCat[j] contient le nombre de catégories pour la variable j

##### PredPoisson:
- dist[i, j] contient les loi de prédiction d'une loi de poisson obtenue à l'aide de l'inférence Bayesienne. Le i correspond au mode de déplacement et le j correspond à la variable à analyser
- columns[j] contient les symboles associés aux colonnes qui contiennent les variables j

##### PredNormal:
- dist[i, j] contient les loi de prédiction d'une loi Normale obtenue à l'aide de l'inférence Bayesienne. Le i correspond au mode de déplacement et le j correspond à la variable à analyser
- columns[j] contient les symboles associés aux colonnes qui contiennent les variables j

##### Dists:
Cette structure contient les structures des différentes lois de prédiction.<br>

In [9]:
struct Occurences
    mode::Array{Int64}
    variables::Array{Array}
    columns::Array{Symbol}
    nCat::Array{Int64}
end
struct PredPoisson
    dist::Array{NegativeBinomial}
    columns::Array{Symbol}
end
struct PredNormal
    dist::Array{Normal}
    columns::Array{Symbol}
end
struct Dists
    occ::Occurences
    predPoisson::PredPoisson
    predNormal::PredNormal
end

### Fonctions internes ("privées")

#### getOccurences(trainSet, col, nCat)
Cette fonction construit la structure Occurences pour un modèle donné.
- trainSet: l'ensemble des données d'où on va extraire les occurences. 
- col: le tableau qui contient toutes les variables dont il faut extraire les occurences.
- nCat: le nombre de catégories pour chaque variable explicative.

#### getDistributions(trainSet, colCat, nCat, colPoisson, colNormal, paramsNormal)
Cette fonction construit la structure Dists pour des modèles données.
- trainSet: l'ensemble des données d'où on va extraire les distributions
- colCat: le tableau qui contient les symboles des variables de type catégorielle.
- nCat: le nombre de catégories pour chaque variable explicative catégorielle.
- colPoisson: le tableau qui contient les symboles des variables de type Poisson.
- colNormal: le tableau qui contient les symboles des variables de type Normale.
- paramsNormal: tableau qui contient les σ² des variables de type Normale

#### findProb(occ, variable, value)
Cette fonction retourne la probabilité associée à une variable explicative pour une valeur donnée.
- occ: Contient les occurences (n) qui permettent de calculer la probabilité.
- variable: Représente l'indice de la variable pour le champ variables dans occ.
- value représente la valeur de la variable explicative.

#### filterMode(row, occ)
Cette fonction retourne quel mode est le plus probable pour un déplacement. Elle appelle la fonction findProb pour toutes les variables explicatives présentes dans occ.
- row: Le déplacement dont on veut prévoir le mode.
- occ: Contient les occurences (n) qui permettent de calculer la probabilité.


In [70]:
function getOccurences(trainSet::DataFrame, col::Array{Symbol}, nCat::Array{Int64})
    mode = Array{Int64, 1}(undef, k)
    for i=1:k
        mode[i] = size(trainSet[trainSet[:D_Mode] .== i, :], 1)
    end
    
    variables = Array{Array{Int64, 1}, 2}(undef, (k, length(col)))
    
    for i=1:5
        for j=1:length(col)
            variables[i, j] = Array{Int64, 1}(undef, nCat[j])
            for l=1:nCat[j]
                variables[i, j][l] = size(trainSet[(trainSet[:D_Mode] .== i) .& (trainSet[col[j]] .== l), :], 1)
            end
        end
    end
    return Occurences(mode, variables, col, nCat)
end

function getDistributions(trainSet::DataFrame, colCat::Array{Symbol}, nCat::Array{Int64}, colPoisson::Array{Symbol}, colNormal::Array{Symbol}, paramsNormal::Array{Float64, 2})
    predPoisson = Array{NegativeBinomial, 2}(undef, k, length(colPoisson))
    predNormal = Array{Normal, 2}(undef, k, length(colNormal))
    for i=1:k
        for j=1:length(colPoisson)
            data = trainSet[colPoisson[j]][trainSet[:D_Mode] .== i]
            r = sum(data)
            p = 1 - 1/(length(data) + 1)
            predPoisson[i, j] = NegativeBinomial(r, p)
        end
        for j=1:length(colNormal)
            data = trainSet[colNormal[j]][trainSet[:D_Mode] .== i]
            n = length(data)
            predNormal[i, j] = Normal(mean(data), sqrt((n + 1)/n * paramsNormal[j, i]))
        end
    end
    return Dists(getOccurences(trainSet, colCat, nCat), PredPoisson(predPoisson, colPoisson), PredNormal(predNormal, colNormal))
end

function findProb(occ::Occurences, variable::Int64, value::Int64)
    q = Array{Float64}(undef, k)
    for i=1:k
        q[i] = log(occ.variables[i, variable][value] + 1) - log(occ.mode[i] + length(occ.nCat[variable]))        
    end
    return q
end

function findProb(predPoisson::PredPoisson, variable::Int64, value::Int64)
    q = Array{Float64}(undef, k)
    for i=1:k
        q[i] = logpdf(predPoisson.dist[i, variable], value)      
    end
    return q
end

function findProb(predNormal::PredNormal, variable::Int64, value)
    q = Array{Float64}(undef, k)
    for i=1:k
        q[i] = logpdf(predNormal.dist[i, variable], value)      
    end
    return q
end

function filterMode(row::DataFrameRow, dists::Dists)
    p = Array{Float64}(undef, k)
    for i=1:k
       p[i] = log(dists.occ.mode[i] + 1) - log(sum(dists.occ.mode) + length(dists.occ.mode)) 
    end 
    for i=1:length(dists.occ.columns)
        x̃ = row[dists.occ.columns[i]]
        p = p + findProb(dists.occ, i, x̃)
    end
    for i=1:length(dists.predPoisson.columns)
        x̃ = row[dists.predPoisson.columns[i]]
        p = p + findProb(dists.predPoisson, i, x̃)
    end
    for i=1:length(dists.predNormal.columns)
        x̃ = row[dists.predNormal.columns[i]]
        p = p + findProb(dists.predNormal, i, x̃)
    end
    
     _, ind = findmax(p)
    
    return ind
end

filterMode (generic function with 1 method)

### Fonction "publique"

##### getPercentRightBayes(colCat, nCat, colPoisson, colNormal, paramsNormal)
Calcule la validité d'un modèle
- colCat: le tableau qui contient les symboles des variables de type catégorielle.
- nCat: le nombre de catégories pour chaque variable explicative catégorielle.
- colPoisson: le tableau qui contient les symboles des variables de type Poisson.
- colNormal: le tableau qui contient les symboles des variables de type Normale.
- paramsNormal: tableau qui contient les σ² des variables de type Normale

In [71]:
function getPercentRightBayes(colCat::Array{Symbol}, nCat::Array{Int64}, colPoisson::Array{Symbol}, colNormal::Array{Symbol}, paramsNormal::Array{Float64, 2})
    set = getTestSet(train)
    dists = getDistributions(set.train, colCat, nCat, colPoisson, colNormal, paramsNormal)
    sizeTest = size(set.test, 1)
    est = Array{Int64}(undef, sizeTest)
    for i=1:sizeTest
        est[i] = filterMode(set.test[i, :], dists)
    end
    output = validate(est, set.test[:D_Mode])
    print(output.output)
end

getPercentRightBayes (generic function with 1 method)

## Exemple

In [72]:
getPercentRightBayes(    
    [:P_PERMIS, :D_MOTIF, :P_STATUT],
    [5, 13, 8],
    [:M_AUTO],
    [:M_dist_centre],
    [[150.0 100.0 100.0 150.0 150.0];]
)

Voiture : 82.53624170166644%
Transport Collectif : 37.96169630642955%
Hybride : 0.0%
Actif : 28.25788751714678%
Autre : 82.48803827751196%
Total : 63.945680875141456%


### Ajout d'une nouvelle variable, la municipalité

In [92]:
d = Array{Int64}(undef,n)

for i=1:length(train[:M_DOMSM])
    mun = train[:M_DOMSM][i];
    if mun == 101 || mun == 102
        d[i] = 1;
    elseif (mun >= 103 && mun <= 112) || (mun >= 120 && mun <= 124)
        d[i] = 2;
    elseif mun >= 113 && mun <= 118
        d[i] = 3;
    elseif mun >= 126 && mun <= 141
        d[i] = 4;
    elseif mun >= 301 && mun <= 310
        d[i] = 5;
    elseif mun >= 401 && mun <= 408
        d[i] = 6;
    elseif mun >= 501 && mun <= 576
        d[i] = 7;
    elseif mun >= 601 && mun <= 691
        d[i] = 8;
    else
        d[i] = 9;
    end
end

train[:M_DOMSM_G] = d

getPercentRightBayes(    
    [:P_PERMIS,  :D_MOTIF, :P_STATUT, :M_DOMSM_G],
    [5, 13, 8, 9],
    [:M_AUTO],
    [:M_dist_centre],
    [[165.1401487395247 73.87513909433788 108.07524856463094 156.13987012660058 168.0994914221992];]
)

Voiture : 76.6518600777346%
Transport Collectif : 55.71377714489142%
Hybride : 0.0%
Actif : 25.01405283867341%
Autre : 80.23483365949119%
Total : 63.01018483591098%
